In [1]:
import numpy as np
import tensorflow as tf
import tensorflow_probability as tfp
from numpy.linalg import norm

2022-04-15 21:47:37.493378: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-15 21:47:37.493417: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [2]:
#!pip install tensorflow
#!pip install tensorflow_probability

In [3]:
k = 20
n = 100
N1 = tf.constant(tf.random.normal([k,n], 0, 1, tf.float32, seed=1))
N2 = tf.constant(tf.random.normal([k,n], 0, 1, tf.float32, seed=1))

2022-04-15 21:47:39.621066: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-15 21:47:39.621100: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-15 21:47:39.621127: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (dell-Inspiron-5448): /proc/driver/nvidia/version does not exist
2022-04-15 21:47:39.621433: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
def make_constraint():
    def constraint(L):
        return tf.linalg.band_part(L, -1, 0)
    return constraint

In [5]:
constraint = make_constraint()

In [12]:
total = int((k**2+k)/2)
l = tf.random.normal([total],0,1,tf.float32, seed=1)
L1 = tf.Variable(tfp.math.fill_triangular(l),constraint=constraint)
l = tf.random.normal([total],0,1,tf.float32, seed=1)
L2 = tf.Variable(tfp.math.fill_triangular(l),constraint=constraint)
# diag = np.ones(k)
# for i in range(k):
#     diag[i] + i
# L1 = tf.linalg.set_diag(L1,diag)
# L2 = tf.linalg.set_diag(L2,diag)
b = 0.5
def loss_fn():
    X1 = L1@N1
    X2 = L2@N2
    X1 = tf.math.l2_normalize(X1, 1)
    X2 = tf.math.l2_normalize(X2, 1)
    C = tf.matmul(X1,X2,transpose_b=True)
    D = tf.math.square(tf.math.subtract(C,b))
    # - tf.linalg.trace(tf.math.log(L1)) - tf.linalg.trace(tf.math.log(L2))
    # - tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L1))) - tf.reduce_sum(tf.math.log(tf.linalg.diag_part(L2)))
    return tf.reduce_sum(D) 

In [13]:
losses = tfp.math.minimize(loss_fn,
                           num_steps=10000,
                           optimizer=tf.optimizers.Adam(learning_rate=0.1))

In [14]:
losses

<tf.Tensor: shape=(10000,), dtype=float32, numpy=
array([104.23366 ,  99.27533 ,  94.33078 , ...,   7.10001 ,   7.10001 ,
         7.100011], dtype=float32)>

In [15]:
sigma1 = tf.matmul(L1,L1,transpose_b=True).numpy()
sigma2 = tf.matmul(L2,L2,transpose_b=True).numpy()

In [16]:
N = 10000
x1 = np.random.multivariate_normal(np.zeros(k),sigma1,N)
x2 = np.random.multivariate_normal(np.zeros(k),sigma2,N)
a = norm(x1,axis=1)
b = norm(x2,axis=1)
a = a.reshape(N,1)
b = b.reshape(N,1)
M = a@b.T
cos_sim = np.divide(x1@x2.T,M)
np.mean(abs(cos_sim.flatten()))

/tmp/ipykernel_11919/2125472353.py:2: RuntimeWarning: covariance is not positive-semidefinite.
  x1 = np.random.multivariate_normal(np.zeros(k),sigma1,N)
/tmp/ipykernel_11919/2125472353.py:3: RuntimeWarning: covariance is not positive-semidefinite.
  x2 = np.random.multivariate_normal(np.zeros(k),sigma2,N)


0.5689783689886951